In [14]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox, Output, Layout

# Constants
hbar = 1.055e-34  # Reduced Planck constant (Joule-seconds)
m_e = 9.11e-31  # Electron rest mass (kg)
eV_to_J = 1.602e-19  # 1 eV in Joules

# Effective mass options (in terms of electron mass m_e)
def effective_mass_electron(x):
    return (0.067 + 0.083 * x) * m_e

def effective_mass_hole(x, hole_type):
    if hole_type == "heavy":
        return (0.62 + 0.14 * x) * m_e
    elif hole_type == "light":
        return (0.15 + 0.09 * x) * m_e

# Initial layer configuration
layers = [
    (100, "GaAs"),
    *[(90, "AlGaAs"), (10, "GaAs")] * 5,
    (300, "AlGaAs"), (200, "GaAs"), (300, "AlGaAs"), (100, "GaAs"),
    (300, "AlGaAs"), (70, "GaAs"), (300, "AlGaAs"), (50, "GaAs"),
    (300, "AlGaAs"), (30, "GaAs"), (300, "AlGaAs"), (20, "GaAs"),
    (300, "AlGaAs"), (50, "GaAs")
]

# Functions for calculations
def calculate_gaas_bandgap(T):
    return 1.57 - 0.057 * (1 + 2 / (np.exp(240 / T) - 1))

def conduction_band_offset(x):
    if x < 0.41:
        return 0.79 * x
    else:
        return 0.475 - 0.335 * x + 0.143 * (x ** 2)

def valence_band_offset(x):
    return -0.46 * x

def calculate_band_edges(material, gaas_bandgap, x_algaas, gaas_hole_type, algaas_hole_type):
    if material == "GaAs":
        upper = gaas_bandgap / 2
        lower = -gaas_bandgap / 2
    elif material == "AlGaAs":
        upper = (gaas_bandgap / 2) + conduction_band_offset(x_algaas)
        lower = -(gaas_bandgap / 2) + valence_band_offset(x_algaas)
    elif material == "AlAs":
        x_algaas = 1
        upper = (gaas_bandgap / 2) + conduction_band_offset(x_algaas)
        lower = -(gaas_bandgap / 2) + valence_band_offset(x_algaas)
    else:
        raise ValueError("Unknown material: " + material)
    return upper, lower

def calculate_energy_levels(n_max, width, effective_mass):
    d = width * 1e-10
    levels = [(hbar ** 2 * (np.pi ** 2) * n ** 2) / (2 * effective_mass * d ** 2) / eV_to_J for n in range(1, n_max + 1)]
    return levels

def generate_potential_plot(T, x_algaas, n_max, manual_y_range, y_min, y_max, manual_x_range, x_min, x_max, gaas_hole_type, algaas_hole_type):
    gaas_bandgap = calculate_gaas_bandgap(T)

    x = []
    upper_potential = []
    lower_potential = []
    energy_levels = []

    current_x = 0
    n_colors = ["red", "blue", "green", "orange", "purple"]

    for width, material in layers:
        upper, lower = calculate_band_edges(material, gaas_bandgap, x_algaas, gaas_hole_type, algaas_hole_type)
        x_region = np.linspace(current_x, current_x + width, 100)
        upper_region = np.full_like(x_region, upper)
        lower_region = np.full_like(x_region, lower)
        x.extend(x_region)
        upper_potential.extend(upper_region)
        lower_potential.extend(lower_region)
        
        if material in ["GaAs", "AlGaAs", "AlAs"]:
            conduction_mass = effective_mass_electron(x_algaas if material == "AlGaAs" else (1 if material == "AlAs" else 0))
            hole_type = gaas_hole_type if material == "GaAs" else algaas_hole_type
            valence_mass = effective_mass_hole(x_algaas if material == "AlGaAs" else (1 if material == "AlAs" else 0), hole_type)
            conduction_levels = calculate_energy_levels(n_max, width, conduction_mass)
            valence_levels = calculate_energy_levels(n_max, width, valence_mass)
            energy_levels.append((current_x, conduction_levels, valence_levels))
        current_x += width

    fig, ax = plt.subplots(figsize=(15, 5))
    ax.plot(x, upper_potential, color='black', label='Upper Potential Limit')
    ax.plot(x, lower_potential, color='black', label='Lower Potential Limit')

    for pos, conduction_levels, valence_levels in energy_levels:
        for i, (E_c, E_v) in enumerate(zip(conduction_levels, valence_levels), 1):
            ax.hlines(E_c, pos, pos + width, color=n_colors[(i - 1) % len(n_colors)], linestyle='--')
            ax.hlines(-E_v, pos, pos + width, color=n_colors[(i - 1) % len(n_colors)], linestyle='-')

    if manual_y_range:
        ax.set_ylim(y_min, y_max)
    if manual_x_range:
        ax.set_xlim(x_min, x_max)

    ax.set_xlabel('Position (Å)', fontsize=12)
    ax.set_ylabel('Energy (eV)', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# UI Widgets
T_slider = widgets.FloatSlider(value=300, min=1, max=500, step=10, description='T (K)')
x_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description='Al-Fraction (x)')
n_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Quantum Number')
manual_y_toggle = widgets.ToggleButton(value=True, description='Manual Y-Range?')
y_min_box = widgets.FloatText(value=-1.2, description='Y-Min (eV)')
y_max_box = widgets.FloatText(value=1.2, description='Y-Max (eV)')
manual_x_toggle = widgets.ToggleButton(value=True, description='Manual X-Range?')
x_min_box = widgets.FloatText(value=0, description='X-Min (Å)')
x_max_box = widgets.FloatText(value=3225, description='X-Max (Å)')
gaas_hole_dropdown = widgets.Dropdown(options=["heavy", "light"], value="heavy", description='GaAs Hole')
algaas_hole_dropdown = widgets.Dropdown(options=["heavy", "light"], value="heavy", description='AlGaAs/AlAs Hole')

# Layer management UI
layer_output = Output()

def display_layers():
    with layer_output:
        layer_output.clear_output()
        print("Current Layers:")
        for i, (width, material) in enumerate(layers):
            print(f"{i+1}: {material}, {width} Å")

def add_layer(material, thickness):
    layers.append((thickness, material))
    display_layers()

def reset_layers():
    global layers
    layers = [
    (100, "GaAs"),
    *[(90, "AlGaAs"), (10, "GaAs")] * 5,
    (300, "AlGaAs"), (200, "GaAs"), (300, "AlGaAs"), (100, "GaAs"),
    (300, "AlGaAs"), (70, "GaAs"), (300, "AlGaAs"), (50, "GaAs"),
    (300, "AlGaAs"), (30, "GaAs"), (300, "AlGaAs"), (20, "GaAs"),
    (300, "AlGaAs"), (50, "GaAs")
]
    display_layers()

def remove_all_layers():
    layers.clear()
    display_layers()

material_dropdown = widgets.Dropdown(options=["GaAs", "AlGaAs", "AlAs"], value="GaAs", description="Material")
thickness_input = widgets.IntText(value=100, description="Thickness (Å)")
add_layer_button = widgets.Button(description="Add Layer")
reset_button = widgets.Button(description="Reset Layers")
remove_all_button = widgets.Button(description="Remove All Layers")

add_layer_button.on_click(lambda b: add_layer(material_dropdown.value, thickness_input.value))
reset_button.on_click(lambda b: reset_layers())
remove_all_button.on_click(lambda b: remove_all_layers())

def update_plot_ui(T, x_algaas, n_max, manual_y_range, y_min, y_max, manual_x_range, x_min, x_max, gaas_hole_type, algaas_hole_type):
    generate_potential_plot(T, x_algaas, n_max, manual_y_range, y_min, y_max, manual_x_range, x_min, x_max, gaas_hole_type, algaas_hole_type)

controls = VBox([
    HBox([T_slider, x_slider]),
    HBox([n_slider, gaas_hole_dropdown, algaas_hole_dropdown]),
    HBox([manual_y_toggle, y_min_box, y_max_box]),
    HBox([manual_x_toggle, x_min_box, x_max_box]),
])

layer_controls = VBox([
    HBox([material_dropdown, thickness_input, add_layer_button]),
    HBox([reset_button, remove_all_button]),
    layer_output
])

out = widgets.interactive_output(update_plot_ui, {
    'T': T_slider,
    'x_algaas': x_slider,
    'n_max': n_slider,
    'manual_y_range': manual_y_toggle,
    'y_min': y_min_box,
    'y_max': y_max_box,
    'manual_x_range': manual_x_toggle,
    'x_min': x_min_box,
    'x_max': x_max_box,
    'gaas_hole_type': gaas_hole_dropdown,
    'algaas_hole_type': algaas_hole_dropdown
})

display(VBox([controls, out, layer_controls]))
reset_layers()
